In [34]:
import pandas as pd
import numpy as np

In [35]:
train_interactions_matrix = np.load('Data/train_rating_matrix.npy', allow_pickle=False)
train_interactions_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [36]:
from lightfm.data import Dataset
dataset = Dataset()
dataset.fit(np.arange(train_interactions_matrix.shape[0]), np.arange(train_interactions_matrix.shape[1]))

In [37]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 138493, num_items 26744.


In [38]:
from scipy.sparse import coo_matrix
# Convert to COO format
coo_interactions_matrix = coo_matrix(train_interactions_matrix)

In [39]:
test_interactions_matrix = np.load('Data/test_rating_matrix.npy', allow_pickle=False)

In [40]:
test_coo_interactions_matrix = coo_matrix(test_interactions_matrix)

In [41]:
test_coo_interactions_matrix

<138493x26744 sparse matrix of type '<class 'numpy.int8'>'
	with 2441485 stored elements in COOrdinate format>

In [56]:
from lightfm import LightFM

model = LightFM(loss='warp')
model.fit(interactions=coo_interactions_matrix, epochs=100, num_threads=12,verbose=True)

Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 100%|██████████| 100/100 [05:41<00:00,  3.42s/it]


In [57]:
%%time
prediction=model.predict(np.full(num_items,4500), np.arange(num_items))
print(prediction.argmax(),prediction.max())

46 3.0180802
CPU times: user 2.37 ms, sys: 0 ns, total: 2.37 ms
Wall time: 1.8 ms


In [58]:
print("Training Interactions Shape:", coo_interactions_matrix.shape)
print("Test Interactions Shape:", test_coo_interactions_matrix.shape)


Training Interactions Shape: (138493, 26744)
Test Interactions Shape: (138493, 26744)


In [59]:
from lightfm.evaluation import auc_score
train_auc = auc_score(model=model, test_interactions=test_coo_interactions_matrix,num_threads=12)


In [60]:
print(f"test_auc = {train_auc.mean():.2f}")

test_auc = 0.99


In [61]:
from lightfm.evaluation import precision_at_k, recall_at_k
precision = precision_at_k(model, test_interactions=test_coo_interactions_matrix,train_interactions=coo_interactions_matrix, k=50,num_threads=12).mean()
recall= recall_at_k(model,test_interactions= test_coo_interactions_matrix,train_interactions=coo_interactions_matrix, k=50,num_threads=12).mean()
print(f"precision = {precision:.2f} \t recall = {recall:.2f}")

precision = 0.11 	 recall = 0.40


In [62]:
from lightfm.evaluation import reciprocal_rank
reciprocal_rank = reciprocal_rank(model, test_interactions=test_coo_interactions_matrix,train_interactions=coo_interactions_matrix,num_threads=12).mean()

In [63]:
reciprocal_rank

0.43999815